In [ ]:
!pip install transformers
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.1 MB/s eta 0:00:00


In [ ]:
!pip install simpletransformers
!pip freeze | grep simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━

Tamil Data

In [ ]:
import pandas as pd
df=pd.read_csv('/content/tam_3_train.tsv', delimiter='\t') 
df=df[["text", "category"]]
df = df.dropna()
df_dev=pd.read_csv('/content/tam_3_dev.tsv', delimiter='\t') 
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.dropna()
df_test=pd.read_csv('/content/tam_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'text'}) 
df_test = df_test.dropna()

In [ ]:
tedf=pd.read_csv('/content/eng_tam_train.tsv', delimiter='\t') 
tedf=tedf[["text", "category"]]
tedf = tedf.dropna()
tedf_dev=pd.read_csv('/content/eng_tam_dev.tsv', delimiter='\t') 
tedf_dev=tedf_dev[["text", "category"]]
tedf_dev = tedf_dev.dropna()
tedf_test=pd.read_csv('/content/eng_tam_test.tsv', delimiter='\t') 
tedf_test = tedf_test.dropna()

### Mixing english and tamil translated to tamil data

In [ ]:
train = pd.concat([df, tedf])
valid = pd.concat([df_dev,tedf_dev])
test = pd.concat([tedf_test,df_test])

In [ ]:
print(len(train))
print(len(valid))
print(len(test))

5817
1456
991


In [ ]:
concatenated = pd.concat([train, test,valid])

import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (4958, 2)
Validation set shape: (1653, 2)
Test set shape: (1653, 2)


In [ ]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

Non-anti-LGBT+ content    4319
Homophobic                 518
Transphobic                121
Name: category, dtype: int64
Non-anti-LGBT+ content    1440
Homophobic                 173
Transphobic                 40
Name: category, dtype: int64
Non-anti-LGBT+ content    1440
Homophobic                 173
Transphobic                 40
Name: category, dtype: int64


In [ ]:
train_df['category'] = train_df['category'].replace('Non-anti-LGBT+ content', 'ally')
val_df['category'] = val_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')

In [ ]:
train_df['category'] = train_df['category'].replace('ally', 0)
train_df['category'] = train_df['category'].replace('Homophobic', 1)
train_df['category'] = train_df['category'].replace('Transphobic', 2)

In [ ]:
val_df['category'] = val_df['category'].replace('ally', 0)
val_df['category'] = val_df['category'].replace('Homophobic', 1)
val_df['category'] = val_df['category'].replace('Transphobic', 2)

In [ ]:
test_df['category'] = test_df['category'].replace('ally', 0)
test_df['category'] = test_df['category'].replace('Homophobic', 1)
test_df['category'] = test_df['category'].replace('Transphobic', 2)

In [ ]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=79c3d139891c96ccae113585d789906a20c2155bc8d8052e67b9c666360a4bcd
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [ ]:
import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
train_df

,text,labels
1889,அம்மா நான் உங்களை ஒரு நாளும் ஆண் என சொல்ல விரு...,0
3020,கேமிங் போதை பற்றி கூறுங்கள் அண்ணா,0
883,"அன்புள்ள ஐயா, எனது கருத்து நகைச்சுவையாக இருந்த...",0
2846,கண்ணீருடன்_சிரிக்கும்_முகம் அற்புதமான கதை,0
694,மீரா மிதுனுடன் ஒப்பிட வேண்டாம். மீரா முகம் சுள...,0
...,...,...
240,மை வைத்தல் மாந்திரிகம் பத்தி பேசுங்க,0
499,அவர்கள் ஏன் ஸ்மூச் செய்யவில்லை?! அவர்கள் இருவர...,0
631,"Ayy Ar ஆமாம் அன்பே,உங்களை நினைத்து கொண்டே ஒரு...",0
2419,எங்க போனாலும் இவளுங்க தொல்லைதான்யா.இரத்தம் வர ...,0


## Training Model with Englis Data

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 16,
             "eval_batch_size": 16,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 2e-5}

# Create a ClassificationModel
model = ClassificationModel(
    "albert", "ai4bharat/indic-bert",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.weight', 'predictions.bias', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [ ]:
# Train the model
model.train_model(train_df)

  0%|          | 0/4958 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/310 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/310 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/310 [00:00<?, ?it/s]

(930, 0.40426227665676545)

Validation Set


In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1653 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

{'mcc': 0.3322263360211914,
 'f1': 0.4397250092902267,
 'acc': 0.8777979431336963,
 'eval_loss': 0.33321012545806855}

Testing With Tamil Data

In [ ]:
test_df

,text,labels
1173,அவள் அழகாக இருக்கிறாள்,0
790,MyNameIsRam நன்றி bro red_heart,0
2410,உரக்கச் சொல் உன் உண்மையே ...........கேட்குரவன்...,0
470,"சிஸ்ஸி, உன்னிடம் சிறந்த மனப்பான்மை, நம்பிக்கை ...",1
362,சங்கிகளை எதிர்க்கும் குயர்கள் red_heart fire,0
...,...,...
739,எனது கருத்தை விரும்பு,0
923,"முதலில், நான் ஒன்று தெரிந்து கொள்ள விரும்புகிற...",1
156,கலாச்சார சீர்கேடு ஏற்பட வாய்ப்பு உள்ளது..........,1
1269,அருமையான பேச்சு மாலினி..... லவ் யூ லாட்.... நா...,0


In [ ]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1653 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"சிஸ்ஸி, உன்னிடம் சிறந்த மனப்பான்மை, நம்பிக்கை ...",1,0
1,சோதோம் & கொமோராவின் பாவம்,1,0
2,எக்கா மாலினி உங்க அப்பன் ஒரு ஆம்பளையை பார்த்து...,1,0
3,உங்கள் அப்பா உன்னை மாதிரி நினைத்து ஒரு ஆம்பளை ...,1,0
4,டை தேவுடியா நாயா எங்க நாட்டை வீட்டு செல்,1,0
...,...,...,...
181,சுத்தி நிக்கற அத்தன பேரும் சோ்ந்தா இதுகள மெரட...,0,1
182,Anchor ku பேச தெரியல ....,1,0
183,"நான் ஓரினச்சேர்க்கையில் உடன்படவில்லை, ஏனென்றால...",1,0
184,ஓரினச்சேர்க்கை இயற்கைக்கு மாறானது என்றாலும் தி...,1,0


In [ ]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.91      0.98      0.94      1440
  homophobic       0.57      0.34      0.43       173
 transphobic       0.00      0.00      0.00        40

    accuracy                           0.89      1653
   macro avg       0.49      0.44      0.46      1653
weighted avg       0.85      0.89      0.87      1653



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with Tamil and testing with english

In [ ]:
import pandas as pd
o_test=pd.read_csv('/content/tam_eng_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
e_test = e_test.rename(columns={'text                        ': 'text'})
#e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [ ]:
test_ml = pd.concat([o_test,e_test])
test_ml

,Unnamed: 0,text,category,ln
0,0.0,When each commanta was read and brought up.,Non-anti-LGBT+ content,tam_eng
1,1.0,Bro live a prosperous bouquet OK_hand OK_hand ...,Non-anti-LGBT+ content,tam_eng
2,2.0,Jesus is a step sister,Non-anti-LGBT+ content,tam_eng
3,3.0,Eyes are confused....,Non-anti-LGBT+ content,tam_eng
4,4.0,Come together Gay mean not sex it's love also ...,Non-anti-LGBT+ content,tam_eng
...,...,...,...,...
985,NaN,Looks like Karthik took advantage on Varun whe...,Non-anti-LGBT+ content,NaN
986,NaN,i am really crying pro😢😢😢😭😭😭😭😭😭 ...,Non-anti-LGBT+ content,NaN
987,NaN,They may be transgender but don't ever forgot ...,Non-anti-LGBT+ content,NaN
988,NaN,It is their own choice. I support them No doub...,Non-anti-LGBT+ content,NaN


In [ ]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [ ]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"Similarly, in all matters, the law will come a...",1,0
1,pranesh G you are a transgender.,2,0
2,Let natural calamities like Tsunami and Earthq...,1,0
3,Do the people in your house agree to this type...,1,0
4,Then do not eat by mouth,1,0
...,...,...,...
233,gay culture does not suit the Indian culture. ...,1,0
234,"I'm sharing my experience pls read it, two tra...",2,0
235,"First of all, i wanna know onething. Do you th...",1,0
236,madam nalini....u talking too much right....ok...,1,0


In [ ]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.87      1.00      0.93      1579
  homophobic       1.00      0.01      0.02       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.62      0.34      0.32      1819
weighted avg       0.86      0.87      0.81      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with Tamil and testing with Hindi

In [ ]:
o_test=pd.read_csv('/content/eng_hin_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_hin_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [ ]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [ ]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [ ]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"आप दोनों जिन्होंने मेरी टिप्पणी का जवाब दिया, ...",1,0
1,"सत्य नारायणन , इसे आज भी पश्चिमी दुनिया में प्...",1,0
2,मुझे लगता है..यह मानसिक बीमारी है। जाओ और डॉक्...,1,0
3,पुलिस को उन्हें सजा देनी चाहिए,1,0
4,निर्देशक सर pls गे रिपेंट अगुरा मारी फिल्म एडुंगा,1,0
...,...,...,...
235,वह बहुत अच्छा काम करती है। मांचूर में डेढ़ साल...,2,0
236,मैं आपकी राय मानता हूं। अगर समलैंगिकता गलत है ...,1,0
237,इसी महीने कुत्तों को देखते ही गोली मार दी गई,2,1
238,सुंदर वाणी मधुर वाणी। सवाल का जवाब बहुत अच्छा ...,1,0


In [ ]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.89      0.98      0.93      1579
  homophobic       0.43      0.20      0.27       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.44      0.39      0.40      1819
weighted avg       0.82      0.87      0.84      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with Tamil and testing with Telugu

In [ ]:
o_test=pd.read_csv('/content/eng_tel_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_tel_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [ ]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [ ]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [ ]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,నా వ్యాఖ్యకు ప్రత్యుత్తరం ఇచ్చిన మీరిద్దరూ దీన...,1,0
1,"సత్య నారాయణన్ , నేటికీ పాశ్చాత్య ప్రపంచంలో దీన...",1,0
2,నా అభిప్రాయం .. ఇది మానసిక వ్యాధి. వెళ్లి వైద్...,1,0
3,పోలీసులు వారిని శిక్షించాలి,1,0
4,డైరెక్టర్ సర్ ప్లీస్ గే పశ్చాత్తాపపడండి అగురా ...,1,0
...,...,...,...
238,మీరు తెలివిగా మాట్లాడాలని అనుకుంటున్నారు...,1,0
239,ఆమె చాలా చక్కగా నటిస్తుంది. మంచూర్‌లో డబ్బులు ...,2,0
240,నేను మీ అభిప్రాయాన్ని అంగీకరిస్తున్నాను. స్వలి...,1,0
241,ఈ నెలలో కుక్కలను చూసి కాల్చి చంపారు,2,1


In [ ]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.88      0.98      0.93      1579
  homophobic       0.41      0.13      0.20       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.43      0.37      0.38      1819
weighted avg       0.81      0.87      0.83      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with Tamil and testing with Malyalam

In [ ]:
o_test=pd.read_csv('/content/eng_ml_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_ml_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [ ]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [ ]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [ ]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [ ]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,എന്റെ അഭിപ്രായത്തിന് മറുപടി നൽകിയ നിങ്ങൾ രണ്ടു...,1,0
1,"സത്യ നാരായണൻ , ഇന്നും പാശ്ചാത്യലോകത്ത് ഇത് പ്ര...",1,0
2,എനിക്ക് തോന്നുന്നു .. മാനസിക രോഗമാണ്. പോയി ഡോക...,1,0
3,പോലീസ് അവരെ ശിക്ഷിക്കണം,1,0
4,"സംവിധായകൻ സാർ സ്വവർഗാനുരാഗി പശ്ചാത്തപിക്കട്ടെ,...",1,0
...,...,...,...
249,നിങ്ങൾ ബുദ്ധിപരമായി സംസാരിക്കാൻ വിചാരിക്കുന്നു...,1,0
250,അവൾ വളരെ ഭംഗിയായി അഭിനയിക്കുന്നു. മഞ്ചൂരിൽ സർക...,2,0
251,താങ്കളുടെ അഭിപ്രായം ഞാൻ അംഗീകരിക്കുന്നു. സ്വവർ...,1,0
252,ഈ മാസം നായ്ക്കളെ കണ്ടപ്പോൾ വെടിവച്ചു,2,0


In [ ]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.88      0.98      0.93      1579
  homophobic       0.30      0.09      0.14       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.86      1819
   macro avg       0.39      0.36      0.35      1819
weighted avg       0.79      0.86      0.82      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
